The goal of the homework is to study, based on actual production data, the notion of "peak oil" introduced in lecture 1

#### Administrativia
Groups composition to be provided no later than Wednesday, Sep. 25, 8 PM (by email)\
GROUPS SHALL DIFFER IN MEMBERSHIP FROM HOMEWORK #1\
Homework to be turned in Thursday, Oct. 3, 10 PM (deposit .ipynb file in Moodle / General Informations, only 1 submission per group)\
Please indicate here the names of the group's members:

#### Data set
One CSV file, Oil_production.csv taken from https://www.energyinst.org/statistical-review/resources-and-data-downloads \
Description: crude oil production in million tonnes by country/region (col.A) and year\
NOTE: this is a good example of using "real world" imperfect data sets which have not been prepared for machine processing\
IMPORTANT: Your program shall take as input this csv file, without manual cleaning/preparation

In [68]:
import pandas as pd
import numpy as np

#### Data preparation
Study the data set and identify what cleanup/preparation steps are needed \
Remove all "Total" lines and remove USSR (it appears after the 1991 breakup as its member countries)

In [69]:
# if everything is correct, the csv will be converted into the dataframe
# ";" is the delimiter, separating rows. First 2 rows are skipped, since they are unnecessary. 
# The "," will count as decimal
oildata = pd.read_csv("Oil_production.csv", delimiter=';', skiprows=2, decimal=",")
# the first column(country names) will count as the index
oildata.set_index(oildata.columns[0], inplace=True)
# get rid of rows with N/A values. This effectively removes the last text rows
oildata = oildata.dropna()
# get rid of the USSR row
oildata = oildata.drop("USSR")
# get rid of rows that have "total" in their index's names.
oildata = oildata[~oildata.index.str.contains('Total')]
# the rows after "Total World" didn't get cleaned, so I had to get rid of them by slicing. Crude, but effective
oildata = oildata[0:56]
# replace "^" and "?" with 0, since "^" and "?" represent values and percentages less than 0.05, which are negligible
# now how to replace "-"? With "nan", or with 0? Very difficult question. I guess with NaN. 
# And countries with NaN values can still be kept, since not all of their values are NaN
# besides. This is maths-friendly np.nan, which means that it won't interfere with calculations, ideally
oildata.replace("^", 0, inplace=True)
oildata.replace("-", np.nan, inplace=True)
oildata.replace("?", 0, inplace=True)
# print the first five and the last five rows
print(oildata.head(5))
print(oildata.tail(5))

            1965   1966   1967   1968   1969   1970   1971   1972   1973  \
Country                                                                    
Canada      43,9   48,2   52,7   57,1   62,2   70,1   75,2   86,7  100,3   
Mexico      18,1   18,5   20,5   21,9     23   24,2   24,1   25,1   25,9   
US         427,7  454,5  484,2  502,9  511,4  533,5  525,9  527,9  514,7   
Argentina   13,8   14,6     16   17,5   18,1     20   21,6   22,2   21,6   
Brazil         5    6,2    7,8    8,5    9,3    8,8    9,2      9    9,2   

            1974  ...   2017   2018   2019   2020   2021   2022   2023  \
Country           ...                                                    
Canada      94,4  ...  236,6  257,7  263,5    252  266,6  273,9  277,9   
Mexico      32,4  ...  109,5  102,3   94,9   95,1   96,4   97,6  102,5   
US         491,4  ...  574,2  669,6    751  713,3  715,9  762,1  827,1   
Argentina   21,1  ...   27,3   27,6   33,8   33,7   37,6   45,4   51,2   
Brazil       9,6  ...  

#### Data processing
Write a function which takes as input :
1. time series (floats)
2.   a threshold (float)

and returns true if the last two data points of the time series are more than threshold % lower than the maximum value in the series

In [9]:
# the series is a list of floating values extracted from the dataframe, corresponding to years 1965-2023
# the threshold is a float between 0 and 1. 
# Higher threshold means that the last two values must be much smaller in order for the function to return "True"
def has_peaked(series, threshold):
    # by default, the boolean is false
    bool = False
    # checks if the last two points are smaller than the maximum by at least the threshold percentage
    # for example, if threshold percentage is 0.2, then we check if the last points are smaller than max*(1-0.2)
    if series[-1]< series.max*(1-threshold) and series[-2]<series.max*(1-threshold):
        bool = True
    return bool
# returns bool

##### First graph
Plot on Y axis the percentage (0 to 1) of all oil-producing countries whose recent production (last 2 years) is more than threshold % lower than their peak as a function of threshold, with threshold ranging from 0 to 100 (X axis)\
How many countries have not peaked yet?

#### Second graph
Plot a stacked vertical bar graph with each country on the X axis (sorted by decreasing value of the country's maximum production). Each bar displays both the 2023  production and the maximum production (in a lighter color) of the country\
Bonus points if the bar's color encodes the oil-producing region the country belongs to.